Test Phase4_3d file

In [9]:
# install anaconda: https://www.anaconda.com/products/individual
# if use Visual Studio Code, install python extension and select the python interpreter that includes anaconda
# install micropip: https://pypi.org/project/micropip/. You can also run anaconda prompt and type "pip install micropip"
import numpy as np
import micropip
await micropip.install("scipy")

In [10]:
from Phase4_trace_3d import Phase4_trace_3d
# from generateTestPositions import generateTestPositions
import generateTestPositions
from generateTestPositions import get_velocity_function
from generateTestPositions import generateTestPositions
from generateTestPositions import get_initial_position
# Cell 4
# Testing with dummy data
NumOfDataPoints = 1
clusterness = 0.1 # smaller number the more clustered
# Input conditions
initial_positions = []
v = []
# for i in range(1):
#     [x,y,z] = get_initial_position(i)
#     initial_positions.append([x, y, z])
#     vel = get_velocity_function(i)
#     v.append(vel)
initial_positions = np.zeros((NumOfDataPoints,3))
initial_positions[0] = get_initial_position(1)
v.append(lambda t: [np.sin(t), np.cos(t), 1])
noise = 1e-3
theta_degrees = 1.8
rev = 2  # revolutions of camera for the entire process
NOS = int(rev * 360 / theta_degrees)
NOS = 53
NOS_per_section = 30  # must be larger than 5 to satisfy equations
camera_speed = 0.5  # in Hz or revolution per second
SRD = 1  # m, Source-Reference Distance
RDD = 1  # m, Reference-Detector (screen) Distance
radius = 1

method = 'acceleration'
dataPiling = 'overlap'

# Auto-calculations of the rest of the parameters derived from the setting above
delta_T = camera_speed * theta_degrees / 360
shots_per_second = 1 / delta_T

# Define the velocity function
# v = lambda t: [0.9 * np.sin(t), 0.9 * np.cos(t), 1]

# Pack conditions into a list
conditions = [noise, delta_T, NOS, theta_degrees, NOS_per_section, SRD, RDD,method,dataPiling]
print(conditions)

xz_proj = np.zeros((NOS, NumOfDataPoints*2))
real_positions = np.zeros((NOS, NumOfDataPoints*3))
# Generate test positions
for i in range(NumOfDataPoints):
    vel = v[i]
    xz_proj[:,i*2:i*2+2], real_positions[:,i*3:i*3+3]= generateTestPositions(vel, initial_positions[i], conditions)

[0.001, 0.0025, 53, 1.8, 30, 1, 1, 'acceleration', 'overlap']


In [11]:
# Print the first few rows of the generated data
print("xz_proj:")
print(xz_proj[:])
print("real_positions:")
print(real_positions[:])

xz_proj:
[[3.99743654e-01 1.27196125e-04]
 [4.00803791e-01 4.83402780e-03]
 [4.03670100e-01 1.02525124e-02]
 [4.03435032e-01 1.48830699e-02]
 [4.05025055e-01 1.98060627e-02]
 [4.06780634e-01 2.56680449e-02]
 [4.07761435e-01 3.06291222e-02]
 [4.08683302e-01 3.63348351e-02]
 [4.10457166e-01 4.17353582e-02]
 [4.11129668e-01 4.68666975e-02]
 [4.12201656e-01 5.12628145e-02]
 [4.12292064e-01 5.74838542e-02]
 [4.13342372e-01 6.29910556e-02]
 [4.14511593e-01 6.93149926e-02]
 [4.14989360e-01 7.44263828e-02]
 [4.15344797e-01 7.93147432e-02]
 [4.16041657e-01 8.52856867e-02]
 [4.16455485e-01 9.11709959e-02]
 [4.18090584e-01 9.70431908e-02]
 [4.17321093e-01 1.01582034e-01]
 [4.17125288e-01 1.07825562e-01]
 [4.16499736e-01 1.14099105e-01]
 [4.16588144e-01 1.20401014e-01]
 [4.15749211e-01 1.25793944e-01]
 [4.14664674e-01 1.32750360e-01]
 [4.15048002e-01 1.38879305e-01]
 [4.14650919e-01 1.44836685e-01]
 [4.13393669e-01 1.51279569e-01]
 [4.12833017e-01 1.57483572e-01]
 [4.10486021e-01 1.63094508e-01]
 

In [12]:
import pandas as pd
import os

def rename_files_replace_space(directory_path):
    # Get a list of all files in the directory
    filenames = os.listdir(directory_path)
    
    for filename in filenames:
        # Replace spaces with underscores
        new_filename = filename.replace('_', '')
        
        # Construct the full old and new file paths
        old_filepath = os.path.join(directory_path, filename)
        new_filepath = os.path.join(directory_path, new_filename)
        
        # Rename the file
        os.rename(old_filepath, new_filepath)

In [13]:
# Import the function directly from the module
from Phase4_trace_3d import Phase4_trace_3d
from smooth_points import smooth_points
from particle_path_finder import ParticlePathFinder as pf
# from particle_path_finder import OriginalPathKeeper as opk
import random
import numpy as np
import pandas as pd
import os


# # read data from .csv file
# input_data = pd.read_csv('particle_tracking_input.csv')

# columnIndex = 0

theta_degrees = 1.8
# # Ai recognition caller
alpha = np.radians(theta_degrees)  # Example rotation angle in radians
path_finder = pf(alpha)
# original_path_keeper = opk(alpha)
# iterate through each row of the input data as a snapshot, and put the snapshot into the particle path finder
# rename_files_replace_space("Data_3particles")

# print(os.listdir("Data_3particles"))
sorted_filenames = sorted(os.listdir("Data_3particles"), key=lambda x: int(x.split('Shot')[1].split('.csv')[0]))
print(sorted_filenames)

for file in sorted_filenames:
    if file.endswith(".csv"):
        filename = os.path.join("Data_3particles", file)
        input_data = pd.read_csv(filename, header=None)
        input_data = np.array(np.transpose(input_data))
        values =  input_data[0]
        print("values:", values)
        print("read file: ", filename)

        paired_values = []
        i = 0
        for j in range(len(values)//2):
            
            inputList = values[i:i+2]
           
            # input format, list of tuple of two elements (x,y)
            paired_values.append(inputList)

            # scambled_values = random.shuffle(paired_values.copy())
            # print("paired values:", paired_values)
            
            i += 2

        print("paired values:", paired_values)
        path_finder.append(paired_values)
    
    

    

# estimated_positions = np.zeros((NOS,3*NumOfDataPoints))
# # Then you can call it just by its name:
# for i in range(NumOfDataPoints):
#     estimated_positions_single = Phase4_trace_3d(conditions, xz_proj[:,i*2:i*2+2])
#     estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

#     estimated_positions[:,i*3:i*3+3] = estimated_positions_single
sorted_particle_data = path_finder.get_particle_data()
print("sorted_positions: ", sorted_particle_data)



['Shot0.csv', 'Shot1.csv', 'Shot2.csv', 'Shot3.csv', 'Shot4.csv', 'Shot5.csv', 'Shot6.csv', 'Shot7.csv', 'Shot8.csv', 'Shot9.csv', 'Shot10.csv', 'Shot11.csv', 'Shot12.csv', 'Shot13.csv', 'Shot14.csv', 'Shot15.csv', 'Shot16.csv', 'Shot17.csv', 'Shot18.csv', 'Shot19.csv', 'Shot20.csv', 'Shot21.csv', 'Shot22.csv', 'Shot23.csv', 'Shot24.csv', 'Shot25.csv', 'Shot26.csv', 'Shot27.csv', 'Shot28.csv', 'Shot29.csv', 'Shot30.csv', 'Shot31.csv', 'Shot32.csv', 'Shot33.csv', 'Shot34.csv', 'Shot35.csv', 'Shot36.csv', 'Shot37.csv', 'Shot38.csv', 'Shot39.csv', 'Shot40.csv', 'Shot41.csv', 'Shot42.csv', 'Shot43.csv', 'Shot44.csv', 'Shot45.csv', 'Shot46.csv', 'Shot47.csv', 'Shot48.csv', 'Shot49.csv', 'Shot50.csv', 'Shot51.csv', 'Shot52.csv', 'Shot53.csv']
values: [243.5  97.5 294.5  77.5 183.5 111.5]
read file:  Data_3particles\Shot0.csv
paired values: [array([243.5,  97.5]), array([294.5,  77.5]), array([183.5, 111.5])]
current_snapShotIndex:  0
current shot:  [array([243.5,  97.5]), array([294.5,  77.5

In [17]:
# print("sorted_positions type: ", type(sorted_particle_data[0]['coords']))
print("sorted_positions: ", np.array(sorted_particle_data[0]['coords']))

# for i in range(NumOfDataPoints):
#     estimated_positions_single = Phase4_trace_3d(conditions, xz_proj[:,i*2:i*2+2])
#     estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

#     estimated_positions[:,i*3:i*3+3] = estimated_positions_single
estimated_positions = np.zeros((NOS,3*NumOfDataPoints))
for i in range(NumOfDataPoints):
    estimated_positions_single = Phase4_trace_3d(conditions, np.array(sorted_particle_data[i]['coords']))
    estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

    estimated_positions[:,i*3:i*3+3] = estimated_positions_single
# estimated_positions_single = Phase4_trace_3d(conditions, np.array(sorted_particle_data[0]['coords']))

# estimated_positions = estimated_positions_single

print('estimated positions: \n',estimated_positions)
# print('real positions: \n',real_positions)

sorted_positions:  [[243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.   97.5]
 [244.   97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [244.   97.5]
 [243.5  97.5]
 [244.   97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]
 [243.5  97.5]]
NOS:  53
NOS_per_Section:  30
estimated positions: 
 [[ 0.10643521 -0.88062463  0.0532392 ]
 [ 0.13999836 -0.8899156   0.05430135]
 [ 0.17274094 -0.89788692  0.0551231 ]


In [15]:
import matplotlib
print(matplotlib.get_backend())
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')  # or another interactive backend

# from mpl_toolkits.mplot3d import Axes3D
import os


def Phase4Graph(real_positions, estimated_positions, conditions, v, f2):
    noise, delta_T, NOS, theta_degree, N, SRD, RDD, method, dataPiling = conditions
    
    plt.ion()
    # Ensure that real_positions and estimated_positions are the same length
    real_positions = real_positions[:len(estimated_positions)]
    
    fig = plt.figure(f2, figsize=(20, 16), dpi=100)
    
    ax1 = fig.add_subplot(111, projection='3d')
    
    for i in range(NumOfDataPoints):
        ax1.plot(real_positions[:, i*3], real_positions[:, i*3+1], real_positions[:, i*3+2], 'r', markersize=10)
        ax1.plot(estimated_positions[:, i*3], estimated_positions[:, i*3+1], estimated_positions[:, i*3+2], 'b', markersize=20)
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_xlim([-0.5, 0.5])
    ax1.set_ylim([-0.5, 0.5])
    ax1.set_zlim([0, 1.5])
    ax1.set_title('Real Positions vs Estimated Positions in 3D')
    ax1.legend(['Real Positions', 'Estimated Positions'])
    
    distances = np.linalg.norm(real_positions - estimated_positions, axis=1)
    min_val, min_idx = np.min(distances), np.argmin(distances)
    max_val, max_idx = np.max(distances), np.argmax(distances)
    
    # ax1.text(*real_positions[min_idx], f'Min ({min_idx}): {real_positions[min_idx]}', color='r')
    # ax1.text(*estimated_positions[min_idx], f'Min ({min_idx}): {estimated_positions[min_idx]}', color='b')
    # ax1.text(*real_positions[max_idx], f'Max ({max_idx}): {real_positions[max_idx]}', color='r')
    # ax1.text(*estimated_positions[max_idx], f'Max ({max_idx}): {estimated_positions[max_idx]}', color='b')
    
    print(f'Noise: {noise}\nMin Distance: {min_val}\nMax Distance: {max_val}\nNumber of Shots: {NOS}\nRotation: {theta_degree}\nTime Between Shots: {delta_T}\nNOS Per Section: {N}\nMethod: {method}\nData Piling: {dataPiling}')
    
    v_str = str(v).replace('<function ', '').replace(' at 0x' + hex(id(v))[2:] + '>', '')
    
    print(f'True velocity:\n{v_str}\nt is time elapsed')
    
    
    plt.show()


TkAgg


In [16]:
f2 = 0
Phase4Graph(real_positions, estimated_positions, conditions, v, f2)

Noise: 0.001
Min Distance: 0.27776150756450885
Max Distance: 1.0038444284505659
Number of Shots: 53
Rotation: 1.8
Time Between Shots: 0.0025
NOS Per Section: 30
Method: acceleration
Data Piling: overlap
True velocity:
[<lambda> at 0x000001F0A7097060>]
t is time elapsed
